In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [2]:
#reading my table that was updated after Part 1 of assignment
toronto_data = pd.read_csv('toronto_data.csv')
toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [3]:
#reading the geospatial coordinates csv into the notebook
geo_coord = pd.read_csv('Geospatial_Coordinates.csv')
geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
!pip install geocoder

import geocoder

latitude=[]
longitude=[]
for code in toronto_data['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

     |████████████████████████████████| 102kB 5.5MB/s eta 0:00:01
M1B [43.811390000000074, -79.19661999999994]
M1C [43.78574000000003, -79.15874999999994]
M1E [43.765750000000025, -79.17469999999997]
M1G [43.76812000000007, -79.21760999999998]
M1H [43.76944000000003, -79.23891999999995]
M1J [43.74446000000006, -79.23116999999996]
M1K [43.725820000000056, -79.26460999999995]
M1L [43.71289000000007, -79.28505999999999]
M1M [43.72360000000003, -79.23495999999994]
M1N [43.695100000000025, -79.26465999999994]
M1P [43.75998000000004, -79.26939999999996]
M1R [43.75075000000004, -79.30053999999996]
M1S [43.794520000000034, -79.26707999999996]
M1T [43.784910000000025, -79.29721999999998]
M1V [43.817810000000065, -79.28043999999994]
M1W [43.80079000000006, -79.32161999999994]
M1X [43.83412000000004, -79.21667999999994]
M2H [43.80225000000007, -79.35557999999997]
M2J [43.780970000000025, -79.34780999999998]
M2K [43.781120000000044, -79.38059999999996]
M2L [43.756980000000055, -79.38059999999996]


In [5]:
#dataframe of lat/long
toronto_df2= pd.DataFrame([latitude,longitude],['Lantitude','Longitude']).T
toronto_df2

,Lantitude,Longitude
0,43.81139,-79.19662
1,43.78574,-79.15875
2,43.76575,-79.17470
3,43.76812,-79.21761
4,43.76944,-79.23892
5,43.74446,-79.23117
6,43.72582,-79.26461
7,43.71289,-79.28506
8,43.72360,-79.23496
9,43.69510,-79.26466


In [6]:
#combining the main dataframe with the lat/long
toronto_mstr=pd.concat([toronto_data,toronto_df2],axis=1)
toronto_mstr.head()

,Postal Code,Borough,Neighbourhood,Lantitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [7]:
print('Week 3 Assignment - End part 2 of 3')

Week 3 Assignment - End part 2 of 3


In [8]:
print('Week 3 Assignment - Beginning part 3 of 3')

Week 3 Assignment - Beginning part 3 of 3


In [9]:
#Getting lat/Long of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_mstr['Lantitude'], toronto_mstr['Longitude'], toronto_mstr['Borough'], toronto_mstr['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
#Dataframd that only contains boroughs with Toronto in name
toronto_data = toronto_mstr[toronto_mstr['Borough'].astype(str).str.contains('Toronto')]
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Lantitude,Longitude
37,M4E,East Toronto,The Beaches,43.67709,-79.29547
41,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467
43,M4M,East Toronto,Studio District,43.66213,-79.33497
44,M4N,Central Toronto,Lawrence Park,43.72843,-79.38713


In [12]:
CLIENT_ID = 'GFX0WXEI1IX3TXHUL4X0QMCMPRMRIZLDZWEEIGMORMAFTWY3' # your Foursquare ID
CLIENT_SECRET = 'XCBQ355UDDVNK2JT4NPB530XUBW11CWK0F0OM4JKHZARU2RD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GFX0WXEI1IX3TXHUL4X0QMCMPRMRIZLDZWEEIGMORMAFTWY3
CLIENT_SECRET:XCBQ355UDDVNK2JT4NPB530XUBW11CWK0F0OM4JKHZARU2RD


In [18]:
# Gathering first neighborhood that contains Toronto

toronto_data.loc[0, 'Neighbourhood']

KeyError: 0